Create regions file from segment predictions matrices:
    
   #### predictions are saved as:
    
        [audio_id_segm_start_sec_to_end_sec, array shape(n_target classes, number of frames (6 sec segm = 259 timeframes))]
        
        
   #### region file structure:
    
    onset(sec), class_value, duration(sec)
        

for each predictions file :

(NEAT would be to make this as a parallel process that alwys looks into a folder and starts this as the files become available check this: https://askubuntu.com/questions/781799/execution-permission-to-all-files-created-under-a-specific-directory-by-default/781909#781909)
        
        for each row:
            check if any 1 (any positive predictions) and what row it apears in!
               Get start_segment
               for each row identified above:
                   onset0 = start_segment + frame_index 1st 1 transformed to sec
                   class_value = row
                   duration = number frames of continuous 1s, transformed into sec.
                   
                   There maybe other events!
                       identify plasces where seq of 1s started again. . . . .
                  

In [ ]:

import itertools as it
import csv


####################################################
#define which files are going to be processed. each prediction_file corresponds to a whole audio file.
predictions_files = []
################################
# frame to sec?
framesPsec = 
##############################################



for pfile in predictions_files:
    regions_csv = []
    
    #load pfile into np array
    predict_array = np.load(pfile, allow_pickle=True)  
        for segm_id, predict_matrix in predict_array #check shape of each predict_matrix is (3,259)
            
            # no point in continuing if the whole matrix is zeros!
            if predict_matrix == 0:
                    continue
            else:
                #get start segm in sec
                start_sec = segm_id.split('s_to')[0].split('_')[1]
            
                for class_value in range(predict_matrix.shape[0]):
                    seq = predict_matrix[class_value,:]
                    summary_seq = [(k, sum(1 for _ in i)) for k, i in it.groupby(seq)]
                    
                    #example:
                    
                        # >>> seq = [0,0,0,0,0,1,1,1,1,0,1,1,1,1]
                        # >>> summary_seq = [(k,sum(1 for _ in i)) for k, i in it.groupby(seq)]
                        #  [(0, 5), (1, 4), (0, 1), (1, 4)]
                                       
                    
                    summary_seq_with_start_pos = list(range(len(summary_seq)))
                    for i, t in enumerate(summary_seq):
                        if i == 0: #first seq gets position 0
                            summary_seq_with_start_pos[0] = (summary_seq[0],0) 
                        else:  # all others get start position = previous seq start_position+duration!
                            summary_seq_with_start_pos[i] = (summary_seq[i], summary_seq_with_start_pos[i-1][-1] + summary_seq[i-1][1])
#                        
                        # what a mess! 


                        summary_seq_with_start_pos_events_only = [t  for t in summary_seq if t[0][0] ==1]
                        nb_distinct_events = len(summary_seq_with_start_pos_events_only)
            
                        for e in range(nb_distinct_events):
                            onset = start_sec + summary_seq_with_start_pos_events_only[e][-1] * framesPsec #get start position in sec
                            duration = summary_seq_with_start_pos_events_only[e][0][-1] * framesPsec   #get duration in sec
                            
                            regions_csv.append([onset, class_value, duration])
        
        
        
        #Join contiguos events separated due to segmentation !!! maybe is not a problem.. but would be better!
        #save regions_csv for this audiofile:             
        with open(save_file, 'w', newline='') as tfile:
            writer = csv.writer(tfile)
            writer.writerow(regions_csv)
                
                